# Training Regression - Reaction

# Import packages

In [ ]:
import os
import sys

current_path=os.getcwd()
print(current_path)

parent_path=os.path.dirname(current_path)
print(parent_path)

if parent_path not in sys.path:
    sys.path.append(parent_path)

In [ ]:
import pandas as pd
from lightning import pytorch as pl
from pathlib import Path

from chemprop import data, featurizers, models, nn

# Change data inputs here

In [ ]:
import numpy as np
chemprop_dir = Path.cwd().parent
num_workers = 0  # number of workers for dataloader. 0 means using main process for data loading
# smiles_column = 'AAM'
# target_columns = ['lograte']

## Load data

In [ ]:
train_npz_normal = np.load(f'../chemprop/data/RC/directed/barriers_cycloadd/barriers_cycloadd_aam_train_normal.npz', allow_pickle=True)
train_v_normal = train_npz_normal['node_attrs']
train_e_normal = train_npz_normal['edge_attrs']
train_idx_g_normal = train_npz_normal['edge_indices']
train_y_normal = train_npz_normal['ys'] 

val_npz_normal = np.load(f'../chemprop/data/RC/directed/barriers_cycloadd/barriers_cycloadd_aam_val_normal.npz', allow_pickle=True)
val_v_normal = val_npz_normal['node_attrs']
val_e_normal = val_npz_normal['edge_attrs']
val_idx_g_normal = val_npz_normal['edge_indices']
val_y_normal = val_npz_normal['ys'] 

test_npz_normal = np.load(f'../chemprop/data/RC/directed/barriers_cycloadd/barriers_cycloadd_aam_test_normal.npz', allow_pickle=True)
test_v_normal = test_npz_normal['node_attrs']
test_e_normal = test_npz_normal['edge_attrs']
test_idx_g_normal = test_npz_normal['edge_indices']
test_y_normal = test_npz_normal['ys'] 

In [ ]:
train_npz_rc = np.load(f'../chemprop/data/RC/directed/barriers_cycloadd/barriers_cycloadd_aam_train_rc.npz', allow_pickle=True)
train_v_rc = train_npz_rc['node_attrs']
train_e_rc = train_npz_rc['edge_attrs']
train_idx_g_rc = train_npz_rc['edge_indices']
train_y_rc = train_npz_rc['ys'] 

val_npz_rc = np.load(f'../chemprop/data/RC/directed/barriers_cycloadd/barriers_cycloadd_aam_val_rc.npz', allow_pickle=True)
val_v_rc = val_npz_rc['node_attrs']
val_e_rc = val_npz_rc['edge_attrs']
val_idx_g_rc = val_npz_rc['edge_indices']
val_y_rc = val_npz_rc['ys'] 

test_npz_rc = np.load(f'../chemprop/data/RC/directed/barriers_cycloadd/barriers_cycloadd_aam_test_rc.npz', allow_pickle=True)
test_v_rc = test_npz_rc['node_attrs']
test_e_rc = test_npz_rc['edge_attrs']
test_idx_g_rc = test_npz_rc['edge_indices']
test_y_rc = test_npz_rc['ys'] 

In [ ]:
train_npz_super = np.load(f'../chemprop/data/RC/directed/barriers_cycloadd/barriers_cycloadd_aam_train_super.npz', allow_pickle=True)
train_v_super = train_npz_super['node_attrs']
train_e_super = train_npz_super['edge_attrs']
train_idx_g_super = train_npz_super['edge_indices']
train_y_super = train_npz_super['ys'] 

val_npz_super = np.load(f'../chemprop/data/RC/directed/barriers_cycloadd/barriers_cycloadd_aam_val_super.npz', allow_pickle=True)
val_v_super = val_npz_super['node_attrs']
val_e_super = val_npz_super['edge_attrs']
val_idx_g_super = val_npz_super['edge_indices']
val_y_super = val_npz_super['ys'] 

test_npz_super = np.load(f'../chemprop/data/RC/directed/barriers_cycloadd/barriers_cycloadd_aam_test_super.npz', allow_pickle=True)
test_v_super = test_npz_super['node_attrs']
test_e_super = test_npz_super['edge_attrs']
test_idx_g_super = test_npz_super['edge_indices']
test_y_super = test_npz_super['ys'] 

In [ ]:
train_v = np.concatenate((train_v_normal, train_v_rc, train_v_super), axis=0)
train_e = np.concatenate((train_e_normal, train_e_rc, train_e_super), axis=0)
train_idx_g = np.concatenate((train_idx_g_normal, train_idx_g_rc, train_idx_g_super), axis=0)
train_y = np.concatenate((train_y_normal, train_y_rc, train_y_super), axis=0)

In [ ]:
val_v = np.concatenate((val_v_normal, val_v_rc, val_v_super), axis=0)
val_e = np.concatenate((val_e_normal, val_e_rc, val_e_super), axis=0)
val_idx_g = np.concatenate((val_idx_g_normal, val_idx_g_rc, val_idx_g_super), axis=0)
val_y = np.concatenate((val_y_normal, val_y_rc, val_y_super), axis=0)

In [ ]:
test_v = np.concatenate((test_v_normal, test_v_rc, test_v_super), axis=0)
test_e = np.concatenate((test_e_normal, test_e_rc, test_e_super), axis=0)
test_idx_g = np.concatenate((test_idx_g_normal, test_idx_g_rc, test_idx_g_super), axis=0)
test_y = np.concatenate((test_y_normal, test_y_rc, test_y_super), axis=0)

In [ ]:
print(train_idx_g.shape, val_y.shape, test_y.shape)

## Perform data splitting for training, validation, and testing

In [ ]:
train_dset = data.ReactionDataset(train_v, train_e, train_idx_g, train_y)
print(train_dset[0][3])
scaler = train_dset.normalize_targets()
# print(scaler)
print(train_dset[0][3])

val_dset = data.ReactionDataset(val_v, val_e, val_idx_g, val_y)
val_dset.normalize_targets(scaler)
test_dset = data.ReactionDataset(test_v, test_e, test_idx_g, test_y)

## Get ReactionDatasets

In [ ]:
edge_index=train_dset[1][0][-2]
print(f'edge_index: {edge_index}')
reverse_index=train_dset[1][0][-1]
print(f'reverse_index: {reverse_index}')

In [ ]:
import numpy as np

np.arange(6).reshape(-1,2)[:, ::-1].ravel()

## Get dataloaders

In [ ]:
train_loader = data.build_dataloader(train_dset, num_workers=num_workers)
val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)
test_loader = data.build_dataloader(test_dset, num_workers=num_workers, shuffle=False)

# Change Message-Passing Neural Network (MPNN) inputs here

## Message passing

Message passing blocks must be given the shape of the featurizer's outputs.

Options are `mp = nn.BondMessagePassing()` or `mp = nn.AtomMessagePassing()`

In [ ]:
train_v[0].shape[1]

In [ ]:
fdims = (train_v[0].shape[1],train_e[0].shape[1]) # the dimensions of the featurizer, given as (atom_dims, bond_dims).
mp = nn.BondMessagePassing(*fdims)

In [ ]:
print(*fdims)

## Aggregation

In [ ]:
print(nn.agg.AggregationRegistry)

In [ ]:
agg = nn.MeanAggregation()

## Feed-Forward Network (FFN)

In [ ]:
print(nn.PredictorRegistry)

In [ ]:
output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)

In [ ]:
ffn = nn.RegressionFFN(output_transform=output_transform)

## Batch norm

In [ ]:
batch_norm = True

## Metrics

In [ ]:
print(nn.metrics.MetricRegistry)

In [ ]:
metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()] 
# Only the first metric is used for training and early stopping

## Construct MPNN

In [ ]:
use_fa_layer=True

In [ ]:
# mpnn = models.MPNN_1(mp, agg, ffn, batch_norm, metric_list, use_fa_layer)
# mpnn

# Training and testing

## Set up trainer

In [ ]:
# trainer = pl.Trainer(
#     logger=False,
#     enable_checkpointing=True,  # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
#     enable_progress_bar=True,
#     accelerator="auto",
#     devices=1,
#     max_epochs=100,  # number of epochs to train for
# )

## Start training

In [ ]:
# trainer.fit(mpnn, train_loader, val_loader)

## Test results

In [ ]:
# results = trainer.test(mpnn, test_loader)

In [ ]:
# results

In [ ]:
model = models.ReactionMPNN(
    hidden_dim=128,
    edge_dim=32,
    k_jump=3,
    out_dim=1,
    warmup_epochs=2,
    init_lr=1e-4,
    max_lr=1e-3,
    final_lr=1e-4,
)

In [ ]:
trainer = pl.Trainer(
    max_epochs=10,
    accelerator="auto", # Tự động chọn 'gpu' nếu có, nếu không thì 'cpu'
    logger=True, # Bật logging
    enable_progress_bar=True,
)

In [ ]:
# # ===================================================================
# # THAY THẾ TOÀN BỘ PHẦN "Training and testing" BẰNG ĐOẠN MÃ NÀY
# # ===================================================================

# # --- Thiết lập các tham số cho Ensemble ---
# ENSEMBLE_SIZE = 5
# all_test_results = []
# output_dir = Path("./reaction_ensemble_results")
# output_dir.mkdir(exist_ok=True)

# print(f"Bắt đầu huấn luyện ensemble với kích thước = {ENSEMBLE_SIZE}")
# print("-" * 30)

# # --- Bắt đầu vòng lặp huấn luyện Ensemble ---
# for i in range(ENSEMBLE_SIZE):
#     print(f"\n--- Đang huấn luyện mô hình {i+1}/{ENSEMBLE_SIZE} ---")
    
#     # 1. Tạo một mô hình MPNN mới cho mỗi lần lặp để đảm bảo trọng số được khởi tạo lại
#     mpnn = models.MPNNWithSkip(mp, agg, ffn, batch_norm, metric_list)

#     # 2. Tạo một Trainer mới, chỉ định nơi lưu checkpoint cho từng mô hình
#     model_checkpoint_dir = output_dir / f"model_{i}"
#     checkpoint_callback = pl.callbacks.ModelCheckpoint(
#         dirpath=model_checkpoint_dir,
#         monitor="val_loss",
#         mode="min",
#         save_top_k=1,
#         filename='best_model'
#     )
    
#     trainer = pl.Trainer(
#         logger=False,
#         enable_checkpointing=True,
#         callbacks=[checkpoint_callback],
#         enable_progress_bar=True,
#         accelerator="auto",
#         devices=1,
#         max_epochs=200,
#     )
    
#     # 3. Huấn luyện mô hình
#     trainer.fit(mpnn, train_loader, val_loader)
    
#     # 4. Chạy kiểm tra (test) trên mô hình tốt nhất vừa được lưu
#     # và lưu kết quả của lần lặp này
#     print(f"--- Đang kiểm tra mô hình {i+1} ---")
#     best_model_path = checkpoint_callback.best_model_path
#     results = trainer.test(mpnn, test_loader, ckpt_path=best_model_path)
#     all_test_results.append(results[0]) # results là một list, lấy phần tử đầu tiên

# # --- Tổng hợp kết quả ---
# print("\n" + "="*30)
# print("HUẤN LUYỆN ENSEMBLE HOÀN TẤT!")
# print("="*30)

# # Chuyển danh sách kết quả thành DataFrame để dễ tính toán
# results_df = pd.DataFrame(all_test_results)

# print("\n--- Kết quả kiểm tra của từng mô hình ---")
# print(results_df)

# print("\n--- Kết quả trung bình của Ensemble ---")
# print(results_df.mean())